In [63]:
import pandas as pd
import numpy as np 

In [64]:
df = pd.read_csv('spam.csv')
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [65]:
df = df.where(pd.notnull(df))
df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [66]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['type'] = encoder.fit_transform(df.Category)
df

,Category,Message,type
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0
...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,1
5568,ham,Will ü b going to esplanade fr home?,0
5569,ham,"Pity, * was in mood for that. So...any other s...",0
5570,ham,The guy did some bitching but I acted like i'd...,0


In [67]:
x = df['Message']
y = df['type']

In [68]:
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.2, random_state = 3)
print(xtrain.shape, xtest.shape)
print(ytrain.shape, ytest.shape)

(4457,) (1115,)
(4457,) (1115,)


In [69]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer

In [70]:
tf = TfidfVectorizer(min_df = 1, stop_words = 'english', lowercase = True)

xtr = tf.fit_transform(xtrain)
xts = tf.transform(xtest)

ytr = ytrain.astype('int')
yts = ytest.astype('int')

In [71]:
with open('tfidf_vectorizer_vocab.pkl', 'wb') as file:
    pickle.dump(tf, file)

In [72]:
for i in xtr:
    print(i)

  (0, 5413)	0.6198254967574347
  (0, 4456)	0.4168658090846482
  (0, 2224)	0.413103377943378
  (0, 3811)	0.34780165336891333
  (0, 2329)	0.38783870336935383
  (0, 4080)	0.18880584110891163
  (0, 3185)	0.29694482957694585
  (0, 3325)	0.31610586766078863
  (0, 2957)	0.3398297002864083
  (0, 2746)	0.3398297002864083
  (0, 918)	0.22871581159877646
  (0, 1839)	0.2784903590561455
  (0, 2758)	0.3226407885943799
  (0, 2956)	0.33036995955537024
  (0, 1991)	0.33036995955537024
  (0, 3046)	0.2503712792613518
  (0, 3811)	0.17419952275504033
  (0, 407)	0.509272536051008
  (0, 3156)	0.4107239318312698
  (0, 2404)	0.45287711070606745
  (0, 6601)	0.6056811524587518
  (0, 2870)	0.5864269879324768
  (0, 7414)	0.8100020912469564
  (0, 50)	0.23633754072626942
  (0, 5497)	0.15743785051118356
  (0, 1559)	0.2398447378557748
  (0, 6551)	0.20583643502688098
  (0, 251)	0.1946360868217845
  (0, 4386)	0.1596900435289571
  (0, 4342)	0.20719530064279246
  (0, 425)	0.25941073647523133
  (0, 2159)	0.2764148878896782
 

Model Building

In [73]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression()
lr_model.fit(xtr, ytr)

LogisticRegression()

In [74]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(xtr, ytr)

RandomForestClassifier(random_state=42)

In [75]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors=2)
knn_model.fit(xtr, ytr)

KNeighborsClassifier(n_neighbors=2)

In [76]:
from sklearn.naive_bayes import MultinomialNB

nb_model = MultinomialNB()
nb_model.fit(xtr, ytr)

MultinomialNB()

Accuracy

In [77]:
from sklearn.metrics import accuracy_score

In [78]:
ypred = lr_model.predict(xtr)
lr_acc = accuracy_score(ytr,ypred)
lr_acc

0.9670181736594121

In [79]:
ypred = rf_model.predict(xtr)
rf_acc = accuracy_score(ytr,ypred)
rf_acc

1.0

In [80]:
ypred = knn_model.predict(xtr)
knn_acc = accuracy_score(ytr,ypred)
knn_acc

0.9468252187570114

In [81]:
ypred = nb_model.predict(xtr)
nb_acc = accuracy_score(ytr,ypred)
nb_acc

0.9807045097599282

Saving the Modal

In [82]:
import pickle

with open('model.pkl', 'wb') as model_file:
    pickle.dump(rf_model, model_file)

Using the Model

In [83]:
with open('model.pkl', 'rb') as file:
    model = pickle.load(file)

with open('tfidf_vectorizer_vocab.pkl', 'rb') as file:
    tfidf_vectorizer = pickle.load(file)

text_to_check = ["Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"]
text_vectorized = tfidf_vectorizer.transform(text_to_check)

prediction = model.predict(text_vectorized)

if prediction[0] == 1:
    print("spam.")
else:
    print("ham.")

spam.
